# Example 5: Batching with PolyCRTBuilder
Firstly, let's import some modules that we will depend on in the next examples.

In [1]:
import time
import random
import pickle
import threading
import seal
from seal import ChooserEvaluator, \
    Ciphertext, \
    Decryptor, \
    Encryptor, \
    EncryptionParameters, \
    Evaluator, \
    IntegerEncoder, \
    FractionalEncoder, \
    KeyGenerator, \
    MemoryPoolHandle, \
    Plaintext, \
    SEALContext, \
    EvaluationKeys, \
    GaloisKeys, \
    PolyCRTBuilder, \
    ChooserEncoder, \
    ChooserEvaluator, \
    ChooserPoly

In this fundamental example we discuss and demonstrate a powerful technique
called `batching`. If N denotes the degree of the polynomial modulus, and T
the plaintext modulus, then batching is automatically enabled in SEAL if
T is a prime and congruent to 1 modulo 2*N. In batching the plaintexts are
viewed as matrices of size 2-by-(N/2) with each element an integer modulo T.
Homomorphic operations act element-wise between encrypted matrices, allowing
the user to obtain speeds-ups of several orders of magnitude in naively
vectorizable computations. We demonstrate two more homomorphic operations
which act on encrypted matrices by rotating the rows cyclically, or rotate
the columns (i.e. swap the rows). These operations require the construction
of so-called `Galois keys`, which are very similar to evaluation keys.

In [2]:
def print_parameters(context):
    print("/ Encryption parameters:")
    print("| poly_modulus: " + context.poly_modulus().to_string())
    print("| coeff_modulus_size: " + (str)(context.total_coeff_modulus().significant_bit_count()) + " bits")
    print("| plain_modulus: " + (str)(context.plain_modulus().value()))
    print("| noise_standard_deviation: " + (str)(context.noise_standard_deviation()))

parms = EncryptionParameters()
parms.set_poly_modulus("1x^4096 + 1")
parms.set_coeff_modulus(seal.coeff_modulus_128(4096)) 
parms.set_plain_modulus(40961) # Note that 40961 is a prime number and 2*4096 divides 40960.
context = SEALContext(parms)
print_parameters(context)

/ Encryption parameters:
| poly_modulus: 1x^4096 + 1
| coeff_modulus_size: 110 bits
| plain_modulus: 40961
| noise_standard_deviation: 3.19


We can see that batching is indeed enabled by looking at the encryption
parameter qualifiers created by SEALContext.

In [3]:
qualifiers = context.qualifiers()
keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()

We need to create Galois keys for performing matrix row and column rotations.
Like evaluation keys, the behavior of Galois keys depends on a decomposition
bit count. The noise budget consumption behavior of matrix row and column
rotations is exactly like that of relinearization. Thus, we refer the reader
to Example 2 for more details.

Here we use a moderate size decomposition bit count.

In [4]:
gal_keys = GaloisKeys()
keygen.generate_galois_keys(30, gal_keys)

Since we are going to do some multiplications we will also relinearize.

In [5]:
ev_keys = EvaluationKeys()
keygen.generate_evaluation_keys(30, ev_keys)

We also set up an Encryptor, Evaluator, and Decryptor here.

In [6]:
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

Batching is done through an instance of the PolyCRTBuilder class so need
to start by constructing one.

In [7]:
crtbuilder = PolyCRTBuilder(context)

The total number of batching `slots' is degree(poly_modulus). The matrices
we encrypt are of size 2-by-(slot_count / 2).

In [8]:
slot_count = (int)(crtbuilder.slot_count())
row_size = (int)(slot_count / 2)
print("Plaintext matrix row size: " + (str)(row_size))

Plaintext matrix row size: 2048


Printing the matrix is a bit of a pain.

We're not going to print every column of the matrix (there are 2048). Instead
print this many slots from beginning and end of the matrix.

In [9]:
def print_matrix(matrix):
    print("")
    print_size = 5
    current_line = "    ["
    for i in range(print_size):
        current_line += ((str)(matrix[i]) + ", ")
    current_line += ("..., ")
    for i in range(row_size - print_size, row_size):
        current_line += ((str)(matrix[i]))
        if i != row_size-1: current_line += ", "
        else: current_line += "]"
    print(current_line)
    current_line = "    ["
    for i in range(row_size, row_size + print_size):
        current_line += ((str)(matrix[i]) + ", ")
    current_line += ("..., ")
    for i in range(2*row_size - print_size, 2*row_size):
        current_line += ((str)(matrix[i]))
        if i != 2*row_size-1: current_line += ", "
        else: current_line += "]"
    print(current_line)
    print("")

The matrix plaintext is simply given to PolyCRTBuilder as a flattened vector
of numbers of size slot_count. The first row_size numbers form the first row,
and the rest form the second row. Here we create the following matrix:
    [ 0,  1,  2,  3,  0,  0, ...,  0 ]
    [ 4,  5,  6,  7,  0,  0, ...,  0 ]

In [10]:
pod_matrix = [0]*slot_count
pod_matrix[0] = 0
pod_matrix[1] = 1
pod_matrix[2] = 2
pod_matrix[3] = 3
pod_matrix[row_size] = 4
pod_matrix[row_size + 1] = 5
pod_matrix[row_size + 2] = 6
pod_matrix[row_size + 3] = 7
print("Input plaintext matrix:")
print_matrix(pod_matrix)

Input plaintext matrix:

    [0, 1, 2, 3, 0, ..., 0, 0, 0, 0, 0]
    [4, 5, 6, 7, 0, ..., 0, 0, 0, 0, 0]



First we use PolyCRTBuilder to compose the matrix into a plaintext.

In [11]:
plain_matrix = Plaintext()
crtbuilder.compose(pod_matrix, plain_matrix)

Next we encrypt the plaintext as usual.

In [12]:
encrypted_matrix = Ciphertext()
print("Encrypting: ")
encryptor.encrypt(plain_matrix, encrypted_matrix)
print("Done")
print("Noise budget in fresh encryption: " +
      (str)(decryptor.invariant_noise_budget(encrypted_matrix)) + " bits")

Encrypting: 
Done
Noise budget in fresh encryption: 80 bits


Operating on the ciphertext results in homomorphic operations being performed
simultaneously in all 4096 slots (matrix elements). To illustrate this, we
form another plaintext matrix
    [ 1,  2,  1,  2,  1,  2, ..., 2 ]
    [ 1,  2,  1,  2,  1,  2, ..., 2 ]
and compose it into a plaintext.

In [13]:
pod_matrix2 = []
for i in range(slot_count): pod_matrix2.append((i % 2) + 1)
plain_matrix2 = Plaintext()
crtbuilder.compose(pod_matrix2, plain_matrix2)
print("Second input plaintext matrix:")
print_matrix(pod_matrix2)

Second input plaintext matrix:

    [1, 2, 1, 2, 1, ..., 2, 1, 2, 1, 2]
    [1, 2, 1, 2, 1, ..., 2, 1, 2, 1, 2]



We now add the second (plaintext) matrix to the encrypted one using another
new operation -- plain addition -- and square the sum.

In [14]:
print("Adding and squaring: ")
evaluator.add_plain(encrypted_matrix, plain_matrix2)
evaluator.square(encrypted_matrix)
evaluator.relinearize(encrypted_matrix, ev_keys)
print("Done")

Adding and squaring: 
Done


How much noise budget do we have left?

In [15]:
print("Noise budget in result: " + (str)(decryptor.invariant_noise_budget(encrypted_matrix)) + " bits")

Noise budget in result: 53 bits


We decrypt and decompose the plaintext to recover the result as a matrix.

In [16]:
plain_result = Plaintext()
print("Decrypting result: ")
decryptor.decrypt(encrypted_matrix, plain_result)
print("Done")
crtbuilder.decompose(plain_result)
pod_result = [plain_result.coeff_at(i) for i in range(plain_result.coeff_count())]
print("Result plaintext matrix:")
print_matrix(pod_result)

Decrypting result: 
Done
Result plaintext matrix:

    [1, 9, 9, 25, 1, ..., 4, 1, 4, 1, 4]
    [25, 49, 49, 81, 1, ..., 4, 1, 4, 1, 4]



Note how the operation was performed in one go for each of the elements of the
matrix. It is possible to achieve incredible performance improvements by using
this method when the computation is easily vectorizable.

All of our discussion so far could have applied just as well for a simple vector
data type (not matrix). Now we show how the matrix view of the plaintext can be
used for more functionality. Namely, it is possible to rotate the matrix rows
cyclically, and same for the columns (i.e. swap the two rows). For this we need
the Galois keys that we generated earlier.

We return to the original matrix that we started with.

In [17]:
encryptor.encrypt(plain_matrix, encrypted_matrix)
print("Unrotated matrix: ")
print_matrix(pod_matrix)
print("Noise budget in fresh encryption: " +
      (str)(decryptor.invariant_noise_budget(encrypted_matrix)) + " bits")

Unrotated matrix: 

    [0, 1, 2, 3, 0, ..., 0, 0, 0, 0, 0]
    [4, 5, 6, 7, 0, ..., 0, 0, 0, 0, 0]

Noise budget in fresh encryption: 80 bits


Now rotate the rows to the left 3 steps, decrypt, decompose, and print.

In [18]:
evaluator.rotate_rows(encrypted_matrix, 3, gal_keys)
print("Rotated rows 3 steps left: ")
decryptor.decrypt(encrypted_matrix, plain_result)
crtbuilder.decompose(plain_result)
pod_result = [plain_result.coeff_at(i) for i in range(plain_result.coeff_count())]
print_matrix(pod_result)
print("Noise budget after rotation" +
      (str)(decryptor.invariant_noise_budget(encrypted_matrix)) + " bits")

Rotated rows 3 steps left: 

    [3, 0, 0, 0, 0, ..., 0, 0, 0, 1, 2]
    [7, 0, 0, 0, 0, ..., 0, 0, 4, 5, 6]

Noise budget after rotation53 bits


Rotate columns (swap rows), decrypt, decompose, and print.

In [19]:
evaluator.rotate_columns(encrypted_matrix, gal_keys)
print("Rotated columns: ")
decryptor.decrypt(encrypted_matrix, plain_result)
crtbuilder.decompose(plain_result)
pod_result = [plain_result.coeff_at(i) for i in range(plain_result.coeff_count())]
print_matrix(pod_result)
print("Noise budget after rotation: " +
      (str)(decryptor.invariant_noise_budget(encrypted_matrix)) + " bits")

Rotated columns: 

    [7, 0, 0, 0, 0, ..., 0, 0, 4, 5, 6]
    [3, 0, 0, 0, 0, ..., 0, 0, 0, 1, 2]

Noise budget after rotation: 54 bits


Rotate rows to the right 4 steps, decrypt, decompose, and print.

In [20]:
evaluator.rotate_rows(encrypted_matrix, -4, gal_keys)
print("Rotated rows 4 steps right: ")
decryptor.decrypt(encrypted_matrix, plain_result)
crtbuilder.decompose(plain_result)
pod_result = [plain_result.coeff_at(i) for i in range(plain_result.coeff_count())]
print_matrix(pod_result)
print("Noise budget after rotation: " +
      (str)(decryptor.invariant_noise_budget(encrypted_matrix)) + " bits")

Rotated rows 4 steps right: 

    [0, 4, 5, 6, 7, ..., 0, 0, 0, 0, 0]
    [0, 0, 1, 2, 3, ..., 0, 0, 0, 0, 0]

Noise budget after rotation: 53 bits


The output is as expected. Note how the noise budget gets a big hit in the
first rotation, but remains almost unchanged in the next rotations. This is
again the same phenomenon that occurs with relinearization, where the noise
budget is consumed down to some bound determined by the decomposition bit count
and the encryption parameters. For example, after some multiplications have
been performed, rotations might practically for free (noise budget-wise), but
might be relatively expensive when the noise budget is nearly full, unless
a small decomposition bit count is used, which again is computationally costly.